In [23]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Feature Engineering").getOrCreate()

In [24]:
sales = spark.read.csv("sales",inferSchema=True, header=True).where('Description is not null')

In [25]:
sales.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [47]:
sales.show(truncate=False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |22752

In [51]:
intDF = spark.read.parquet("simple-ml-integers/*.parquet")

In [27]:
intDF.show(5)

+----+----+----+
|int1|int2|int3|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
|   7|   8|   9|
+----+----+----+



In [28]:
scaleDF= spark.read.parquet("simple-ml-scaling")

In [29]:
scaleDF.show(5)

+---+--------------+
| id|      features|
+---+--------------+
|  0|[1.0,0.1,-1.0]|
|  1| [2.0,1.1,1.0]|
|  0|[1.0,0.1,-1.0]|
|  1| [2.0,1.1,1.0]|
|  1|[3.0,10.1,3.0]|
+---+--------------+



In [30]:
simpleDF = spark.read.json("simple-ml")

In [31]:
simpleDF.show(5)

+-----+----+------+------------------+
|color| lab|value1|            value2|
+-----+----+------+------------------+
|green|good|     1|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green|good|    15| 38.97187133755819|
|green|good|    12|14.386294994851129|
+-----+----+------+------------------+
only showing top 5 rows



In [19]:
sales.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

#### Main concepts in Spark Pipelines

DataFrame: Spark ML uses DataFrame from Spark SQL as an ML dataset, which can hold a variety of data types. E.g., a DataFrame could have different columns storing text, feature vectors, true labels, and predictions.

Transformer: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms DataFrame with features into a DataFrame with predictions.

Estimator: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.

Pipeline: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

Parameter: All Transformers and Estimators now share a common API for specifying parameters.

##### Transformers
- functions that convert raw data in some way
- create a new interaction variable (from two other variables), to normalize a column, or to simply turn it into a Double to be input into a model
- primarily used in preprocessing or feature generation
- For Example - A tokenizer

https://spark.apache.org/docs/latest/ml-guide.html

In [32]:
from pyspark.ml.feature import Tokenizer

In [33]:
tokens = Tokenizer(inputCol="Description", outputCol="words")

In [34]:
tokenizedDF = tokens.transform(sales)

In [39]:
tokenizedDF.select('words').show(5,truncate=False)

+------------------------------------------+
|words                                     |
+------------------------------------------+
|[white, hanging, heart, t-light, holder]  |
|[white, metal, lantern]                   |
|[cream, cupid, hearts, coat, hanger]      |
|[knitted, union, flag, hot, water, bottle]|
|[red, woolly, hottie, white, heart.]      |
+------------------------------------------+
only showing top 5 rows



#### Estimators for Preprocessing

abstracts the concept of a learning algorithm or any algorithm that fits or trains on data. Technically, an Estimator implements a method fit(), which accepts a DataFrame and produces a Model, which is a Transformer. For example, a learning algorithm such as LogisticRegression is an Estimator, and calling fit() trains a LogisticRegressionModel, which is a Model and hence a Transformer. 
- An example of this type of estimator is the StandardScaler, which scales your input column according to the range of values in that column to have a zero mean and a variance of 1 in each dimension



## converting words into vector of features

https://towardsdatascience.com/countvectorizer-hashingtf-e66f169e2d4e


In [174]:
from pyspark.ml.feature import HashingTF, IDF

tf = HashingTF(inputCol='words', outputCol='rawFeatures', numFeatures=2000)


In [175]:
featuredDF = tf.transform(tokenizedDF)

In [180]:
featuredDF.select('rawFeatures').show(5,truncate=False)

+------------------------------------------------------------+
|rawFeatures                                                 |
+------------------------------------------------------------+
|(2000,[77,152,538,1160,1348],[1.0,1.0,1.0,1.0,1.0])         |
|(2000,[77,729,1495],[1.0,1.0,1.0])                          |
|(2000,[127,446,477,1467,1514],[1.0,1.0,1.0,1.0,1.0])        |
|(2000,[69,231,411,1138,1194,1932],[1.0,1.0,1.0,1.0,1.0,1.0])|
|(2000,[77,291,756,872,1142],[1.0,1.0,1.0,1.0,1.0])          |
+------------------------------------------------------------+
only showing top 5 rows



In [47]:
idf = IDF(inputCol="rawFeatures", outputCol="features")

idf = idf.fit(featuredDF)

In [48]:
idfDF = idf.transform(featuredDF)

In [49]:
idfDF.select('rawFeatures', 'features').show(5,truncate=False)

+------------------------------------------+-------------------------------------------------------------------------------------------------------------------+
|rawFeatures                               |features                                                                                                           |
+------------------------------------------+-------------------------------------------------------------------------------------------------------------------+
|(20,[0,8,12,17,18],[1.0,1.0,1.0,1.0,1.0]) |(20,[0,8,12,17,18],[1.5751975729379886,1.9299098077088723,1.2214739346338364,1.902283741433941,1.5883816795301255])|
|(20,[9,15,17],[1.0,1.0,1.0])              |(20,[9,15,17],[1.2136882300887148,1.5602476682433286,1.902283741433941])                                           |
|(20,[6,7,14,17],[1.0,2.0,1.0,1.0])        |(20,[6,7,14,17],[1.3245382380313124,3.9147213116866175,1.740764403095775,1.902283741433941])                       |
|(20,[9,11,12,14,18],[1.0,2.0,1.0,

## VectorAssembler

VectorAssembler is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, in order to train ML models like logistic regression and decision trees. VectorAssembler accepts the following input column types: all numeric types, boolean type, and vector type. In each row, the values of the input columns will be concatenated into a vector in the specified order.

Examples

Assume that we have a DataFrame with the columns id, hour, mobile, userFeatures, and clicked:

 id | hour | mobile | userFeatures     | clicked
----|------|--------|------------------|---------
 0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0
userFeatures is a vector column that contains three user features. We want to combine hour, mobile, and userFeatures into a single feature vector called features and use it to predict clicked or not. If we set VectorAssembler’s input columns to hour, mobile, and userFeatures and output column to features, after transformation we should get the following DataFrame:

 id | hour | mobile | userFeatures     | clicked | features
----|------|--------|------------------|---------|-----------------------------
 0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0     | [18.0, 1.0, 0.0, 10.0, 0.5]

In [53]:
intDF.show()

+----+----+----+
|int1|int2|int3|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
|   7|   8|   9|
+----+----+----+



In [54]:
from pyspark.ml.feature import VectorAssembler

In [55]:
va = VectorAssembler(inputCols=['int1','int2','int3'], outputCol='features')

In [56]:
vaDF = va.transform(intDF)

In [57]:
vaDF.show()

+----+----+----+-------------+
|int1|int2|int3|     features|
+----+----+----+-------------+
|   1|   2|   3|[1.0,2.0,3.0]|
|   4|   5|   6|[4.0,5.0,6.0]|
|   7|   8|   9|[7.0,8.0,9.0]|
+----+----+----+-------------+



#### Standard Scaler



StandardScaler transforms a dataset of Vector rows, normalizing each feature to have unit standard deviation and/or zero mean. It takes parameters:

withStd: True by default. Scales the data to unit standard deviation.

withMean: False by default. Centers the data with mean before scaling. It will build a dense output, so take care when applying to sparse input.

In [59]:
from pyspark.ml.feature import StandardScaler

In [60]:
ss = StandardScaler(inputCol='features', outputCol='scaledfeatures')

In [61]:
scaledModel = ss.fit(idfDF)

In [62]:
scaledDF = scaledModel.transform(idfDF)

In [65]:
scaledDF.select('features','scaledfeatures').show(2,truncate=False)

+-------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+
|features                                                                                                           |scaledfeatures                                                                                                     |
+-------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+
|(20,[0,8,12,17,18],[1.5751975729379886,1.9299098077088723,1.2214739346338364,1.902283741433941,1.5883816795301255])|(20,[0,8,12,17,18],[2.153758333544012,2.6720472712954866,1.9545207179560422,2.5591640657380412,2.2217718354023432])|
|(20,[9,15,17],[1.2136882300887148,1.5602476682433286,1.90228374

### RFormula
RFormula selects columns specified by an R model formula. Currently we support a limited subset of the R operators, including ‘~’, ‘.’, ‘:’, ‘+’, and ‘-‘. The basic operators are:

~ separate target and terms
+ concat terms, “+ 0” means removing intercept
- remove a term, “- 1” means removing intercept
: interaction (multiplication for numeric values, or binarized categorical values)
. all columns except target
Suppose a and b are double columns, we use the following simple examples to illustrate the effect of RFormula:

y ~ a + b means model y ~ w0 + w1 * a + w2 * b where w0 is the intercept and w1, w2 are coefficients.
y ~ a + b + a:b - 1 means model y ~ w1 * a + w2 * b + w3 * a * b where w1, w2, w3 are coefficients.
RFormula produces a vector column of features and a double or string column of label. Like when formulas are used in R for linear regression, numeric columns will be cast to doubles. As to string input columns, they will first be transformed with StringIndexer using ordering determined by stringOrderType, and the last category after ordering is dropped, then the doubles will be one-hot encoded.

In [58]:
from pyspark.ml.feature import RFormula

In [63]:
simpleDF.show()

+-----+----+------+------------------+
|color| lab|value1|            value2|
+-----+----+------+------------------+
|green|good|     1|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green|good|    15| 38.97187133755819|
|green|good|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     1| 38.97187133755819|
|  red| bad|     2|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|  red|good|    45| 38.97187133755819|
|green|good|     1|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green|good|    15| 38.97187133755819|
|green|good|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     1| 38.97187133755819|
|  red| bad|     2|14.386294994851129|
+-----+----+------+------------------+
only showing top 20 rows



In [73]:
rf = RFormula(formula='lab ~ .')

In [77]:
rfModel = rf.fit(simpleDF)

In [78]:
labelDF = rfModel.transform(simpleDF)

In [79]:
labelDF.select("features", "label").show(truncate=False)

+---------------------------------+-----+
|features                         |label|
+---------------------------------+-----+
|[0.0,1.0,1.0,14.386294994851129] |1.0  |
|[0.0,0.0,8.0,14.386294994851129] |0.0  |
|[0.0,0.0,12.0,14.386294994851129]|0.0  |
|[0.0,1.0,15.0,38.97187133755819] |1.0  |
|[0.0,1.0,12.0,14.386294994851129]|1.0  |
|[0.0,1.0,16.0,14.386294994851129]|0.0  |
|[1.0,0.0,35.0,14.386294994851129]|1.0  |
|[1.0,0.0,1.0,38.97187133755819]  |0.0  |
|[1.0,0.0,2.0,14.386294994851129] |0.0  |
|[1.0,0.0,16.0,14.386294994851129]|0.0  |
|[1.0,0.0,45.0,38.97187133755819] |1.0  |
|[0.0,1.0,1.0,14.386294994851129] |1.0  |
|[0.0,0.0,8.0,14.386294994851129] |0.0  |
|[0.0,0.0,12.0,14.386294994851129]|0.0  |
|[0.0,1.0,15.0,38.97187133755819] |1.0  |
|[0.0,1.0,12.0,14.386294994851129]|1.0  |
|[0.0,1.0,16.0,14.386294994851129]|0.0  |
|[1.0,0.0,35.0,14.386294994851129]|1.0  |
|[1.0,0.0,1.0,38.97187133755819]  |0.0  |
|[1.0,0.0,2.0,14.386294994851129] |0.0  |
+---------------------------------

In [111]:
labelDF.select("features", "label").explain()

== Physical Plan ==
*(1) Project [features#1660 AS features#1670, UDF(lab#194) AS label#1685]
+- *(1) Project [lab#194, UDF(named_struct(onehot_6a4441f02305, UDF(stridx_05d5bf0425f5#1628, 0), value1_double_RFormula_0c537f2b4141, cast(value1#195L as double), value2, value2#196, interaction_7f067c050cfe, UDF(named_struct(stridx_05d5bf0425f5, stridx_05d5bf0425f5#1628, col2, cast(value1#195L as double))), interaction_626f2e09e204, UDF(named_struct(stridx_05d5bf0425f5, stridx_05d5bf0425f5#1628, value2, value2#196)))) AS features#1660]
   +- *(1) Project [lab#194, value1#195L, value2#196, UDF(color#193) AS stridx_05d5bf0425f5#1628]
      +- *(1) FileScan json [color#193,lab#194,value1#195L,value2#196] Batched: false, Format: JSON, Location: InMemoryFileIndex[file:/E:/BigDataAnalytics/Notebooks/Notebooks/data/simple-ml], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<color:string,lab:string,value1:bigint,value2:double>


In [99]:
rf1 = RFormula(formula='lab~color+value1+value2')

In [101]:
rf1DF = rf1.fit(simpleDF).transform(simpleDF)

In [105]:
rf1DF.select('color','features','label').show(truncate=False)

+-----+---------------------------------+-----+
|color|features                         |label|
+-----+---------------------------------+-----+
|green|[0.0,1.0,1.0,14.386294994851129] |1.0  |
|blue |[0.0,0.0,8.0,14.386294994851129] |0.0  |
|blue |[0.0,0.0,12.0,14.386294994851129]|0.0  |
|green|[0.0,1.0,15.0,38.97187133755819] |1.0  |
|green|[0.0,1.0,12.0,14.386294994851129]|1.0  |
|green|[0.0,1.0,16.0,14.386294994851129]|0.0  |
|red  |[1.0,0.0,35.0,14.386294994851129]|1.0  |
|red  |[1.0,0.0,1.0,38.97187133755819]  |0.0  |
|red  |[1.0,0.0,2.0,14.386294994851129] |0.0  |
|red  |[1.0,0.0,16.0,14.386294994851129]|0.0  |
|red  |[1.0,0.0,45.0,38.97187133755819] |1.0  |
|green|[0.0,1.0,1.0,14.386294994851129] |1.0  |
|blue |[0.0,0.0,8.0,14.386294994851129] |0.0  |
|blue |[0.0,0.0,12.0,14.386294994851129]|0.0  |
|green|[0.0,1.0,15.0,38.97187133755819] |1.0  |
|green|[0.0,1.0,12.0,14.386294994851129]|1.0  |
|green|[0.0,1.0,16.0,14.386294994851129]|0.0  |
|red  |[1.0,0.0,35.0,14.386294994851129]

In [81]:
dataset = spark.createDataFrame(
    [(7, "US", 18, 1.0),
     (8, "CA", 12, 0.0),
     (9, "NZ", 15, 0.0)],
    ["id", "country", "hour", "clicked"])

formula = RFormula(formula="clicked ~   country + hour")

output = formula.fit(dataset).transform(dataset)
output.select("features", "label").show()

+--------------+-----+
|      features|label|
+--------------+-----+
|[0.0,0.0,18.0]|  1.0|
|[0.0,1.0,12.0]|  0.0|
|[1.0,0.0,15.0]|  0.0|
+--------------+-----+



In [78]:
dataset.show()

+---+-------+----+-------+
| id|country|hour|clicked|
+---+-------+----+-------+
|  7|     US|  18|    1.0|
|  8|     CA|  12|    0.0|
|  9|     NZ|  15|    0.0|
+---+-------+----+-------+



#### SQLTransformer
SQLTransformer implements the transformations which are defined by SQL statement. Currently, only supported SQL syntax is like "SELECT ... FROM __THIS__ ..." where "__THIS__" represents the underlying table of the input dataset. The select clause specifies the fields, constants, and expressions to display in the output, and can be any select clause that Spark SQL supports. Users can also use Spark SQL built-in function and UDFs to operate on these selected columns. For example, SQLTransformer supports statements like:

SELECT a, a + b AS a_b FROM __THIS__

SELECT a, SQRT(b) AS b_sqrt FROM __THIS__ where a > 5

SELECT a, b, SUM(c) AS c_sum FROM __THIS__ GROUP BY a, b

Examples

Assume that we have the following DataFrame with columns id, v1 and v2:

 id |  v1 |  v2
----|-----|-----
 0  | 1.0 | 3.0  
 2  | 2.0 | 5.0
This is the output of the SQLTransformer with statement "SELECT *, (v1 + v2) AS v3, (v1 * v2) AS v4 FROM __THIS__":

 id |  v1 |  v2 |  v3 |  v4
----|-----|-----|-----|-----
 0  | 1.0 | 3.0 | 4.0 | 3.0
 2  | 2.0 | 5.0 | 7.0 |10.0

In [80]:
from pyspark.ml.feature import SQLTransformer

df = spark.createDataFrame([
    (0, 1.0, 3.0),
    (2, 2.0, 5.0)
], ["id", "v1", "v2"])
sqlTrans = SQLTransformer(
    statement="SELECT *, (v1 + v2) AS v3, (v1 * v2) AS v4 FROM __THIS__")
sqlTrans.transform(df).explain()

== Physical Plan ==
*(1) Project [id#1232L, v1#1233, v2#1234, (v1#1233 + v2#1234) AS v3#1254, (v1#1233 * v2#1234) AS v4#1255]
+- Scan ExistingRDD[id#1232L,v1#1233,v2#1234]


In [81]:
sqlDF = sqlTrans.transform(df)

In [82]:
sqlDF.show()

+---+---+---+---+----+
| id| v1| v2| v3|  v4|
+---+---+---+---+----+
|  0|1.0|3.0|4.0| 3.0|
|  2|2.0|5.0|7.0|10.0|
+---+---+---+---+----+



### Working with Continuous Features

##### Bucketing
When specifying your
bucket points, the values you pass into splits must satisfy three requirements:
- The minimum value in your splits array must be less than the minimum value in your DataFrame.
- The maximum value in your splits array must be greater than the maximum value in your DataFrame.
- You need to specify at a minimum three values in the splits array, which creates two buckets.

In [201]:
quatiles = sales.approxQuantile('unitprice',[0.2,0.4,0.6,0.8,1.0],0.05)

In [202]:
quatiles

[1.25, 1.95, 2.95, 4.95, 887.52]

In [203]:
quatiles.insert(0,-float("inf"))

In [204]:
quatiles.insert(6,float("inf"))

In [205]:
quatiles

[-inf, 1.25, 1.95, 2.95, 4.95, 887.52, inf]

In [206]:
from pyspark.ml.feature import Bucketizer

b = Bucketizer(splits=quatiles,inputCol='UnitPrice',outputCol='buck_unitprice')

bucketedDF = b.transform(sales)

In [207]:
bucketedDF.select('unitprice','buck_unitprice').sort('buck_unitprice').show(1000)

+---------+--------------+
|unitprice|buck_unitprice|
+---------+--------------+
|     0.42|           0.0|
|     0.85|           0.0|
|     0.85|           0.0|
|     0.72|           0.0|
|     0.95|           0.0|
|     0.99|           0.0|
|     0.85|           0.0|
|     0.85|           0.0|
|     0.84|           0.0|
|     0.95|           0.0|
|     0.85|           0.0|
|     0.43|           0.0|
|     0.85|           0.0|
|     0.85|           0.0|
|     0.85|           0.0|
|     0.95|           0.0|
|     0.85|           0.0|
|     0.42|           0.0|
|     0.85|           0.0|
|     0.43|           0.0|
|     0.85|           0.0|
|     0.85|           0.0|
|     0.85|           0.0|
|     0.42|           0.0|
|     0.43|           0.0|
|     0.42|           0.0|
|     0.42|           0.0|
|     0.85|           0.0|
|     0.42|           0.0|
|     0.65|           0.0|
|     0.42|           0.0|
|     0.55|           0.0|
|     0.85|           0.0|
|     0.85|           0.0|
|

Another option is to split based on percentiles
in our data. This is done with QuantileDiscretizer, which will bucket the values into userspecified buckets with the splits being determined by approximate quantiles values. For instance,
the 90th quantile is the point in your data at which 90% of the data is below that value. You can
control how finely the buckets should be split by setting the relative error for the approximate
quantiles calculation using setRelativeError

In [208]:
from pyspark.ml.feature import QuantileDiscretizer

In [210]:
buckDF = QuantileDiscretizer(numBuckets=5,inputCol='UnitPrice',outputCol='buck_unit')

In [211]:
buckMod = buckDF.fit(sales)

In [214]:
buckMod.getSplits()

[-inf, 1.25, 1.95, 2.95, 5.06, inf]

In [212]:
buckDF = buckMod.transform(sales)

In [213]:
buckDF.select('unitprice','buck_unit').sort('buck_unit').show()

+---------+---------+
|unitprice|buck_unit|
+---------+---------+
|     0.85|      0.0|
|     0.42|      0.0|
|     0.55|      0.0|
|     1.06|      0.0|
|     1.06|      0.0|
|     0.85|      0.0|
|     1.06|      0.0|
|     1.06|      0.0|
|     1.06|      0.0|
|     1.06|      0.0|
|     1.06|      0.0|
|     0.85|      0.0|
|     0.42|      0.0|
|     0.85|      0.0|
|     0.42|      0.0|
|     0.85|      0.0|
|     0.85|      0.0|
|     1.06|      0.0|
|     0.42|      0.0|
|     1.06|      0.0|
+---------+---------+
only showing top 20 rows



### Scaling and Normalization

StandardScaler  --> already covered

MinMaxScaler

The MinMaxScaler will scale the values in a vector (component wise) to the proportional values on a scale from a given min value to a max value. If you specify the minimum value to be 0 and the maximum value to be 1, then all the values will fall in between 0 and 1:


In [219]:
from pyspark.ml.feature import MinMaxScaler

mmscale = MinMaxScaler(min=0,max=1,inputCol='features',outputCol='scaledfeatures')

In [220]:
mmscaleMod = mmscale.fit(labelDF)

In [221]:
mmDF = mmscaleMod.transform(labelDF)

In [225]:
mmDF.select('scaledfeatures').show(truncate=False)

+-----------------------------------------------------------------------------------------------+
|scaledfeatures                                                                                 |
+-----------------------------------------------------------------------------------------------+
|[0.0,1.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.36914560428066195,0.0]                               |
|[0.0,0.0,1.0,0.1590909090909091,0.0,0.0,0.0,0.6666666666666666,0.0,0.0,1.0]                    |
|[0.0,0.0,1.0,0.25,0.0,0.0,0.0,1.0,0.0,0.0,1.0]                                                 |
|[0.0,1.0,0.0,0.3181818181818182,1.0,0.0,0.9375,0.0,0.0,1.0,0.0]                                |
|[0.0,1.0,0.0,0.25,0.0,0.0,0.75,0.0,0.0,0.36914560428066195,0.0]                                |
|[0.0,1.0,0.0,0.3409090909090909,0.0,0.0,1.0,0.0,0.0,0.36914560428066195,0.0]                   |
|[1.0,0.0,0.0,0.7727272727272727,0.0,0.7777777777777778,0.0,0.0,0.36914560428066195,0.0,0.0]    |
|[1.0,0.0,0.0,0.0,1.

In [233]:
mmscale = MinMaxScaler(min=0,max=1,inputCol='rawFeatures',outputCol='scaledfeatures')

In [234]:
mmMod = mmscale.fit(featuredDF)

In [235]:
mmDF1 = mmMod.transform(featuredDF)

In [237]:
mmDF1.select('scaledfeatures').show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------+
|scaledfeatures                                                                                                                                |
+----------------------------------------------------------------------------------------------------------------------------------------------+
|[0.3333333333333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3333333333333333,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.3333333333333333,0.3333333333333333,0.0]|
|[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3333333333333333,0.0,0.0,0.0,0.0,0.0,0.3333333333333333,0.0,0.3333333333333333,0.0,0.0]                |
|[0.0,0.0,0.0,0.0,0.0,0.0,0.3333333333333333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.3333333333333333,0.0,0.0]                               |
|[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3333333333333333,0.0,0.5,0.25,0.0,0.5,0.0,0.0,0.0,0.3333333333333333,0.0]                 

### MaxAbsScaler
The max absolute scaler (MaxAbsScaler) scales the data by dividing each value by the maximum
absolute value in this feature. All values therefore end up between −1 and 1. 

#### Normalizer
The normalizer allows us to scale multidimensional vectors using one of several power norms,
set through the parameter “p”. For example, we can use the Manhattan norm (or Manhattan
distance) with p = 1, Euclidean norm with p = 2, and so on.

In [89]:
from pyspark.ml.feature import Normalizer

nor = Normalizer(p=1,inputCol='rawFeatures',outputCol='norFeatures')

In [90]:
norDF = nor.transform(featuredDF)

In [91]:
norDF.select('norFeatures').show(5,truncate=False)

+-------------------------------------------------------------------------------------------------------------------------+
|norFeatures                                                                                                              |
+-------------------------------------------------------------------------------------------------------------------------+
|(20,[0,8,12,17,18],[0.2,0.2,0.2,0.2,0.2])                                                                                |
|(20,[9,15,17],[0.3333333333333333,0.3333333333333333,0.3333333333333333])                                                |
|(20,[6,7,14,17],[0.2,0.4,0.2,0.2])                                                                                       |
|(20,[9,11,12,14,18],[0.16666666666666666,0.3333333333333333,0.16666666666666666,0.16666666666666666,0.16666666666666666])|
|(20,[2,11,12,16,17],[0.2,0.2,0.2,0.2,0.2])                                                                               |
+-------

In [3]:
weatherDF = spark.read.parquet("ny/*.parquet")

In [8]:
weatherDF.select('Station','Measurement','Year','Values','state','name').show(100)

+-----------+-----------+----+--------------------+-----+--------------------+
|    Station|Measurement|Year|              Values|state|                name|
+-----------+-----------+----+--------------------+-----+--------------------+
|USW00094704|   PRCP_s20|1945|[00 00 00 00 00 0...|   NY|   DANSVILLE MUNI AP|
|USW00094704|   PRCP_s20|1946|[99 46 52 46 0B 4...|   NY|   DANSVILLE MUNI AP|
|USW00094704|   PRCP_s20|1947|[79 4C 75 4C 8F 4...|   NY|   DANSVILLE MUNI AP|
|USW00094704|   PRCP_s20|1948|[72 48 7A 48 85 4...|   NY|   DANSVILLE MUNI AP|
|USW00094704|   PRCP_s20|1949|[BB 49 BC 49 BD 4...|   NY|   DANSVILLE MUNI AP|
|USW00094704|   PRCP_s20|1950|[6E 4B 93 4B BB 4...|   NY|   DANSVILLE MUNI AP|
|USW00094704|   PRCP_s20|1951|[27 4A 32 4A 28 4...|   NY|   DANSVILLE MUNI AP|
|USW00094704|   PRCP_s20|1952|[54 4B 60 4B 6A 4...|   NY|   DANSVILLE MUNI AP|
|USW00094704|   PRCP_s20|1953|[48 4A 37 4A 28 4...|   NY|   DANSVILLE MUNI AP|
|USW00094704|   PRCP_s20|2000|[DE 4A D4 4A CA 4...| 

### Working with Categorical Features
The most common task for categorical features is indexing. Indexing converts a categorical
variable in a column to a numerical one that you can plug into machine learning algorithms.
While this is conceptually simple, there are some catches that are important to keep in mind so
that Spark can do this in a stable and repeatable manner.
In general, we recommend re-indexing every categorical variable when pre-processing just for
consistency’s sake. This can be helpful in maintaining your models over the long run as your
encoding practices may change over time.

StringIndexer

The simplest way to index is via the StringIndexer, which maps strings to different numerical
IDs. Spark’s StringIndexer also creates metadata attached to the DataFrame that specify what
inputs correspond to what outputs

In [15]:
from pyspark.ml.feature import StringIndexer

str_ind_df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"])

In [16]:
str_ind_df.show()

+---+--------+
| id|category|
+---+--------+
|  0|       a|
|  1|       b|
|  2|       c|
|  3|       a|
|  4|       a|
|  5|       c|
+---+--------+



In [17]:
si = StringIndexer(inputCol='category',outputCol='indexed')

In [18]:
siModel = si.fit(str_ind_df)

In [19]:
siDF = siModel.transform(str_ind_df)

In [20]:
siDF.show()

+---+--------+-------+
| id|category|indexed|
+---+--------+-------+
|  0|       a|    0.0|
|  1|       b|    2.0|
|  2|       c|    1.0|
|  3|       a|    0.0|
|  4|       a|    0.0|
|  5|       c|    1.0|
+---+--------+-------+



Using same indexer with other datasets

In [23]:
str_ind_df1 = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "d"), (3, "a"), (4, "a"), (5, "e"),(5, "f"),(5, "f")],
    ["id", "category"])

In [24]:
si.fit(str_ind_df1).transform(str_ind_df1).show()

+---+--------+-------+
| id|category|indexed|
+---+--------+-------+
|  0|       a|    0.0|
|  1|       b|    3.0|
|  2|       d|    4.0|
|  3|       a|    0.0|
|  4|       a|    0.0|
|  5|       e|    2.0|
|  5|       f|    1.0|
|  5|       f|    1.0|
+---+--------+-------+



In [28]:
sales.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows



In [60]:
from pyspark.ml.feature import Tokenizer

In [61]:
tk = Tokenizer(inputCol='Description',outputCol='tokens')

In [62]:
tkDF = tk.transform(sales.select('Description'))

In [75]:
tkDF.show()

+--------------------+--------------------+
|         Description|              tokens|
+--------------------+--------------------+
|WHITE HANGING HEA...|[white, hanging, ...|
| WHITE METAL LANTERN|[white, metal, la...|
|CREAM CUPID HEART...|[cream, cupid, he...|
|KNITTED UNION FLA...|[knitted, union, ...|
|RED WOOLLY HOTTIE...|[red, woolly, hot...|
|SET 7 BABUSHKA NE...|[set, 7, babushka...|
|GLASS STAR FROSTE...|[glass, star, fro...|
|HAND WARMER UNION...|[hand, warmer, un...|
|HAND WARMER RED P...|[hand, warmer, re...|
|ASSORTED COLOUR B...|[assorted, colour...|
|POPPY'S PLAYHOUSE...|[poppy's, playhou...|
|POPPY'S PLAYHOUSE...|[poppy's, playhou...|
|FELTCRAFT PRINCES...|[feltcraft, princ...|
|IVORY KNITTED MUG...|[ivory, knitted, ...|
|BOX OF 6 ASSORTED...|[box, of, 6, asso...|
|BOX OF VINTAGE JI...|[box, of, vintage...|
|BOX OF VINTAGE AL...|[box, of, vintage...|
|HOME BUILDING BLO...|[home, building, ...|
|LOVE BUILDING BLO...|[love, building, ...|
|RECIPE BOX WITH M...|[recipe, b

In [49]:
from pyspark.ml.feature import HashingTF, IDF

In [50]:
tf = HashingTF(inputCol='tokens',outputCol='features')

In [51]:
tfDF = tf.transform(tkDF)

In [52]:
tfDF.select('tokens','features').show(truncate=False)

+------------------------------------------+---------------------------------------------------------------------------+
|tokens                                    |features                                                                   |
+------------------------------------------+---------------------------------------------------------------------------+
|[white, hanging, heart, t-light, holder]  |(262144,[2618,57341,102296,121320,193684],[1.0,1.0,1.0,1.0,1.0])           |
|[white, metal, lantern]                   |(262144,[57341,68281,240983],[1.0,1.0,1.0])                                |
|[cream, cupid, hearts, coat, hanger]      |(262144,[1133,69598,172939,179146,255343],[1.0,1.0,1.0,1.0,1.0])           |
|[knitted, union, flag, hot, water, bottle]|(262144,[4842,42343,61666,149851,167916,187621],[1.0,1.0,1.0,1.0,1.0,1.0]) |
|[red, woolly, hottie, white, heart.]      |(262144,[30600,57341,81060,100086,195459],[1.0,1.0,1.0,1.0,1.0])           |
|[set, 7, babushka, nesting, box

In [9]:
from pyspark.ml.feature import CountVectorizer, Tokenizer

In [10]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

In [13]:
tk = Tokenizer(inputCol='sentence',outputCol='tokens')

In [14]:
tkData = tk.transform(sentenceData)

In [15]:
tkData.show()

+-----+--------------------+--------------------+
|label|            sentence|              tokens|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish Java could...|[i, wish, java, c...|
|  1.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [16]:
cv = CountVectorizer(inputCol='tokens',outputCol='feature')

In [17]:
cvModel = cv.fit(tkData)

In [19]:
cvDF = cvModel.transform(tkData)

In [21]:
tkData.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |tokens                                    |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+



In [22]:
print(cvModel.vocabulary)

['i', 'logistic', 'case', 'heard', 'classes', 'hi', 'regression', 'could', 'are', 'spark', 'about', 'neat', 'java', 'models', 'wish', 'use']


In [91]:
cvDF.select('tokens','feature').show(truncate=False)

+------------------------------------------+------------------------------------------------------+
|tokens                                    |feature                                               |
+------------------------------------------+------------------------------------------------------+
|[white, hanging, heart, t-light, holder]  |(1554,[5,10,17,22,28],[1.0,1.0,1.0,1.0,1.0])          |
|[white, metal, lantern]                   |(1554,[10,19,204],[1.0,1.0,1.0])                      |
|[cream, cupid, hearts, coat, hanger]      |(1554,[56,96,140,176,373],[1.0,1.0,1.0,1.0,1.0])      |
|[knitted, union, flag, hot, water, bottle]|(1554,[13,14,15,47,127,179],[1.0,1.0,1.0,1.0,1.0,1.0])|
|[red, woolly, hottie, white, heart.]      |(1554,[0,10,203,208,212],[1.0,1.0,1.0,1.0,1.0])       |
|[set, 7, babushka, nesting, boxes]        |(1554,[1,45,65,230,236],[1.0,1.0,1.0,1.0,1.0])        |
|[glass, star, frosted, t-light, holder]   |(1554,[17,22,30,54,371],[1.0,1.0,1.0,1.0,1.0])        |


In [181]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish,, Java could use case- classes"),
    (1.0, "Logistic, ,regression ,models are neat")
], ["label", "sentence"])

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)

In [136]:
wordsData.select('words').show(truncate=False)

+---------------------------------------------+
|words                                        |
+---------------------------------------------+
|[hi, i, heard, about, spark]                 |
|[i, wish,,, java, could, use, case-, classes]|
|[logistic,, ,regression, ,models, are, neat] |
+---------------------------------------------+



In [182]:
from pyspark.ml.feature import RegexTokenizer

In [186]:
regex = RegexTokenizer(inputCol='sentence', outputCol='words',pattern='\\W')

In [187]:
regexDF = regex.transform(sentenceData)

In [188]:
regexDF.select('words').show(truncate=False)

+------------------------------------------+
|words                                     |
+------------------------------------------+
|[hi, i, heard, about, spark]              |
|[i, wish, java, could, use, case, classes]|
|[logistic, regression, models, are, neat] |
+------------------------------------------+



In [121]:
tf = HashingTF(inputCol='features',outputCol='newFeatures', numFeatures=100)

In [124]:
tfDF = tf.transform(regexDF)

In [147]:
idfDF.select('features').show()

+--------------------+
|            features|
+--------------------+
|(20,[0,8,12,17,18...|
|(20,[9,15,17],[1....|
|(20,[6,7,14,17],[...|
|(20,[9,11,12,14,1...|
|(20,[2,11,12,16,1...|
|(20,[0,3,13,16],[...|
|(20,[8,10,12,16,1...|
|(20,[3,7,14,17],[...|
|(20,[0,3,11,17],[...|
|(20,[6,14,15],[1....|
|(20,[1,7],[2.2875...|
|(20,[1,7],[4.5750...|
|(20,[5,6,12,15],[...|
|(20,[2,6,9,11],[1...|
|(20,[3,4,6,12,14,...|
|(20,[0,2,3,10,12]...|
|(20,[0,3,10,12],[...|
|(20,[5,11,12,19],...|
|(20,[0,11,12,19],...|
|(20,[0,3,9,10,12]...|
+--------------------+
only showing top 20 rows



### Removing Stop/Common Words

In [189]:
from pyspark.ml.feature import StopWordsRemover

In [190]:
dic = StopWordsRemover.loadDefaultStopWords('english')

In [191]:
stopwords = StopWordsRemover(inputCol='words',stopWords=dic,outputCol='removed_words')

In [192]:
stopedDF = stopwords.transform(regexDF)

In [193]:
stopedDF.select('words','removed_words').show(truncate=False)

+------------------------------------------+------------------------------------+
|words                                     |removed_words                       |
+------------------------------------------+------------------------------------+
|[hi, i, heard, about, spark]              |[hi, heard, spark]                  |
|[i, wish, java, could, use, case, classes]|[wish, java, use, case, classes]    |
|[logistic, regression, models, are, neat] |[logistic, regression, models, neat]|
+------------------------------------------+------------------------------------+



## Creating Word Combinations

In [155]:
from pyspark.ml.feature import NGram

In [156]:
ng = NGram(inputCol='words',n=2, outputCol='ngrams')

In [157]:
ngDF = ng.transform(regexDF)

In [158]:
ngDF.select('words','ngrams').show()

+--------------------+--------------------+
|               words|              ngrams|
+--------------------+--------------------+
|[hi, i, heard, ab...|[hi i, i heard, h...|
|[i, wish,,, java,...|[i wish,,, wish,,...|
|[logistic,, ,regr...|[logistic, ,regre...|
+--------------------+--------------------+



## Converting Words into Numerical Representations

hashingTF, IDF, CountVectorizer, Word2Vec

## Data Summarization

In [197]:
from pyspark.ml.feature import VectorAssembler

In [198]:
va = VectorAssembler(inputCols=['int1','int2','int3'],outputCol='features')

In [199]:
va_intDF = va.transform(intDF)

In [200]:
va_intDF.show()

+----+----+----+-------------+
|int1|int2|int3|     features|
+----+----+----+-------------+
|   1|   2|   3|[1.0,2.0,3.0]|
|   4|   5|   6|[4.0,5.0,6.0]|
|   7|   8|   9|[7.0,8.0,9.0]|
+----+----+----+-------------+



In [212]:
from pyspark.ml.stat import Summarizer

summarizer = Summarizer.metrics("count","mean","variance")

In [213]:
summary = summarizer.summary(va_intDF.features)

In [215]:
va_intDF.select(summary).show(truncate=False)

+---------------------------------+
|aggregate_metrics(features, 1.0) |
+---------------------------------+
|[3, [4.0,5.0,6.0], [9.0,9.0,9.0]]|
+---------------------------------+

